In [27]:
import os
!pip install matplotlib
!pip install numpy
!pip install torch torchvision pillow
!pip install scipy==1.7.3

import sys
import importlib
sys.path.append('DeepRobust/build/lib')
import matplotlib
import matplotlib.pyplot as plt
from deeprobust.image.attack.fgsm import FGSM
import mnist_model as mnist_model
importlib.reload(sys.modules['mnist_model'])
from mnist_model import Net
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR


try:
    from deeprobust.image import optimizer
    from deeprobust.image.defense.fgsmtraining import FGSMtraining
    import deeprobust.image.netmodels.CNN as CNN
    print("Import successful.")
except Exception as e:
    print(f"Import failed: {e}")
import torch
import deeprobust.image.netmodels.train_model as trainmodel
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets


# Define the path to store the dataset
dataset_path = '../data'
model_save_path = 'trained_models/trained_fgsm_model.pth'

cnn = mnist_model.Net()
W = Variable(torch.randn(4, 3), requires_grad=True)
b = Variable(torch.randn(3), requires_grad=True)

optimizer = optim.Adam([W, b])

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

if os.path.exists(os.path.join(os.getcwd(), 'trained_models', 'mnist_cnn.pt')):
    state_dict = torch.load(
        "trained_models/mnist_cnn.pt")
    cnn.load_state_dict(state_dict)
else:
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = Net().to(device)

    # Set training parameters
    batch_size = 64
    test_batch_size = 1000
    epochs = 1
    lr = 1.0
    gamma = 0.7
    log_interval = 10
    dry_run = False

    # Define datasets and loaders
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=True, download=True, transform=transform),
        batch_size=batch_size, shuffle=True
    )
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transform),
        batch_size=test_batch_size, shuffle=False
    )

    # Training setup
    optimizer = optim.Adadelta(model.parameters(), lr=lr)
    scheduler = StepLR(optimizer, step_size=1, gamma=gamma)

    # Train and test
    for epoch in range(1, epochs + 1):
        mnist_model.train(log_interval, dry_run, model, device, train_loader, optimizer, epoch)  # Pass None for args
        mnist_model.test(model, device, test_loader)
        scheduler.step()

    # Save the model
    torch.save(model.state_dict(), "trained_models/mnist_cnn.pt")
    state_dict = torch.load(
        "trained_models/mnist_cnn.pt")
    cnn.load_state_dict(state_dict)


if os.path.exists(model_save_path):
    # Load the pre-trained model weights
    cnn2 = mnist_model.Net()
    cnn2.load_state_dict(torch.load(model_save_path))
    print("Model loaded from disk.")
    train_loader, test_loader = trainmodel.feed_dataset('MNIST', dataset_path)
else:
    # Load the dataset
    train_loader, test_loader = trainmodel.feed_dataset('MNIST', dataset_path)

    # FGSM training
    f = FGSMtraining(cnn, 'cuda')
    defense_model = f.generate(train_loader,test_loader,epoch_num=20)

    # Save the trained model
    torch.save(defense_model.state_dict(), model_save_path)
    cnn2 = mnist_model.Net()
    cnn2.load_state_dict(f.model.state_dict())
    cnn2 = cnn2.to(device=device)
    print("Model trained and saved to disk.")

xx = datasets.MNIST(dataset_path, download=True).data[998:999].to('cpu')
xx = xx.unsqueeze_(1).float() / 255
yy = datasets.MNIST(dataset_path, download=True).targets[998:999].to('cpu')

for i, model in enumerate([cnn, cnn2]):
    if i == 0:
        print("Without Defense:")
    else:
        print("With Defense:")
    F1 = FGSM(model, device=device)  ### or cuda
    AdvExArray = F1.generate(xx, yy, **attack_params['FGSM_MNIST'])

    plt.imshow(AdvExArray.detach()[0].permute(1, 2, 0))
    plt.show()

    predict0 = model(xx)
    predict0 = predict0.argmax(dim=1, keepdim=True)

    predict1 = model(AdvExArray)
    predict1 = predict1.argmax(dim=1, keepdim=True)
    print("original prediction:")
    print(predict0.item())

    print("attack prediction:")
    print(predict1.item())

    print("actual value:")
    print(yy.item())




  Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (61 kB)
Using cached numpy-1.26.4-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (18.2 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.22.4
    Uninstalling numpy-1.22.4:
      Successfully uninstalled numpy-1.22.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
albumentations 1.4.20 requires scipy>=1.10.0, but you have scipy 1.7.3 which is incompatible.
arviz 0.20.0 requires scipy>=1.9.0, but you have scipy 1.7.3 which is incompatible.
jax 0.4.33 requires scipy>=1.10, but you have scipy 1.7.3 which is incompatible.
jaxlib 0.4.33 requires scipy>=1.10, but you have scipy 1.7.3 which is incompatible.
mizani 0.13.0 requires scipy>=1.8.0, but you have scipy 1.7.3 which is incompatible.
plotnine 0.14.1 requires scipy>=1.8.0, but you 

Import successful.
Model loaded from disk.


<ipython-input-27-f34f57e504f3>:47: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(
<ipython-input-27-f34f57e504f3>:97: FutureWarning: You are using `

Without Defense:


NameError: name 'attack_params' is not defined

In [9]:
import os
current_path = os.getcwd()
print(current_path)

/content
